In [9]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf

tf.config.list_physical_devices()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
import numpy as np
from loader.CSVLoader import csv_loader

gt,wave = csv_loader()

name="test8"

all_data_k = np.load(f"./experiment4/{name}_K.npy")
test_data_k = np.load(f'./experiment4/{name}_K_test.npy')

all_data_p = np.load(f"./experiment4/{name}_P.npy")
test_data_p = np.load(f'./experiment4/{name}_P_test.npy')

all_data_Mg = np.load(f"./experiment4/{name}_Mg.npy")
test_data_Mg = np.load(f'./experiment4/{name}_Mg_test.npy')




In [11]:
from creators.CNN3DCreator import  CNN3DNetworkParams
from creators.CRNNCreator import CRNNNetworkParams
from creators.ModelParams import Conv1DParams, Conv2DParams, Conv3DParams, DeepLayerParams, FlattenParams, GenericNeuralNetworkParams, RandomForestParams, RecurrentLayerParams, SVRParams
from creators.CNN1DCreator import  CNN1DNetworkParams
from creators.CNN2DCreator import  CNN2DNetworkParams
from prep.HelperBlocks import pipe, split_3d_to_timeseries
import tensorflow as tf
from prep.MSCBlock import msc_block, msc_block1d

from prep.Normlize import normalize_layers

drop=0.53
epochs=75
optimizer = tf.keras.optimizers.Adam
folds = 5

params_cnn2d = CNN2DNetworkParams(
    conv=[Conv2DParams(kernel=(5,5),pool_size=(2,2),filters=64,normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(3,3),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(2,2),pool_size=(2,2),filters=1024, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: d]), 
        epochs=epochs,
        optimizer=optimizer
        )
)

params_cnn2d_task1 = CNN2DNetworkParams(
    conv=[Conv2DParams(kernel=(4,4),pool_size=(3,3),filters=64,normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(4,4),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(3,3),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: d]), 
        epochs=epochs,
        optimizer=optimizer
        )
)

params_cnn2d_task2 = CNN2DNetworkParams(
    conv=[Conv2DParams(kernel=(5,5),pool_size=(3,3),filters=64,normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(5,5),pool_size=(3,3),filters=512, normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(5,5),pool_size=(2,2),filters=1024, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: d]), 
        epochs=epochs,
        optimizer=optimizer,loss=tf.losses.Huber()
        )
)

In [12]:
from ModelExperiment import ModelExperiment

experiment = ModelExperiment()
experiment.run_experiment(
    params_vector=[params_cnn2d_task2],
    data=[all_data_Mg],
    gt=[gt["Mg"].values],
    folds=folds,
    test_data=[test_data_Mg]
)

Epoch 1/75
44/44 [==============================] - 6s 118ms/step - loss: 158.5455 - mean_absolute_error: 159.0455 - score_metric: 18.8299 - lr: 0.0010 - val_loss: 171.9456 - val_mean_absolute_error: 172.4456 - val_score_metric: 27.7117 - val_lr: 0.0010
Epoch 2/75
44/44 [==============================] - 5s 112ms/step - loss: 157.1197 - mean_absolute_error: 157.6197 - score_metric: 18.0657 - lr: 0.0010 - val_loss: 163.9891 - val_mean_absolute_error: 164.4891 - val_score_metric: 25.2156 - val_lr: 0.0010
Epoch 3/75
44/44 [==============================] - 5s 108ms/step - loss: 154.9127 - mean_absolute_error: 155.4127 - score_metric: 17.8505 - lr: 0.0010 - val_loss: 160.6100 - val_mean_absolute_error: 161.1100 - val_score_metric: 24.4117 - val_lr: 0.0010
Epoch 4/75
44/44 [==============================] - 5s 104ms/step - loss: 150.9569 - mean_absolute_error: 151.4569 - score_metric: 17.3164 - lr: 0.0010 - val_loss: 150.9521 - val_mean_absolute_error: 151.4521 - val_score_metric: 21.2781 -

ResourceExhaustedError: Exception encountered when calling layer "conv2d_29" "                 f"(type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[1024,512,5,5] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

Call arguments received by layer "conv2d_29" "                 f"(type Conv2D):
  • inputs=tf.Tensor(shape=(32, 2, 2, 512), dtype=float32)

In [7]:
res=  np.array(experiment.models[0].predicted_test_values)
print(res.shape)
np.save("./predict/Mg",res)

(1154,)


In [5]:

experiment.models[0].save_history('./experiment4/test6_Mg.csv')
experiment.models[1].save_history('./experiment4/test5_P.csv')
experiment.models[2].save_history('./experiment4/test5_Mg.csv')

IndexError: list index out of range

In [6]:
experiment.get_errors_report('./experiment4/test6_error_Mg.csv')

In [5]:
print(experiment.models[0].crossval_scores)
print(experiment.models[0].crossval_rs)
print(experiment.models[0].mae)

[0.86320454 0.76834422 0.90670264 0.92669553 0.93989766]
[1. 1. 1. 1. 1.]
[0.001 0.001 0.001 0.001 0.001]


In [ ]:
print(experiment.models[5].crossval_scores)
print(experiment.models[5].crossval_rs)
print(experiment.models[5].mae)

[0.92924521 0.93309339 0.90434612 0.91175416 0.97988553]
[0.07072827 0.06497107 0.09564543 0.08582077 0.02011335]
[0.19262939 0.19733602 0.19221746 0.18549364 0.19679249]
